In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt


# Example

In [ ]:
book_example = pd.read_parquet('../input/optiver-realized-volatility-prediction/book_train.parquet/stock_id=0')
trade_example =  pd.read_parquet('../input/optiver-realized-volatility-prediction/trade_train.parquet/stock_id=0')
stock_id = '0'
book_example = book_example[book_example['time_id']==5]
book_example.loc[:,'stock_id'] = stock_id
trade_example = trade_example[trade_example['time_id']==5]
trade_example.loc[:,'stock_id'] = stock_id

In [ ]:
book_example

In [ ]:
plt.plot(book_example['seconds_in_bucket'], book_example['bid_price1'] )  # Plot the chart
plt.show() 

In [ ]:
trade_example.head()

In [ ]:
#Weighted Avg. Price

book_example['wap'] = (book_example['bid_price1'] * book_example['ask_size1'] +
                                book_example['ask_price1'] * book_example['bid_size1']) / (
                                       book_example['bid_size1']+ book_example['ask_size1'])

In [ ]:
book_example

In [ ]:
plt.plot(book_example['seconds_in_bucket'], book_example['wap'] )  # Plot the chart
plt.show() 

In [ ]:
def log_return(list_stock_prices):
    return np.log(list_stock_prices).diff() 

In [ ]:
book_example['log_return'] = log_return(book_example['wap'])
book_example = book_example[~book_example['log_return'].isnull()]

In [ ]:
book_example.head()

In [ ]:
plt.plot(book_example['seconds_in_bucket'], book_example['log_return'] )  # Plot the chart
plt.show() 

In [ ]:
def realized_volatility(series_log_return):
    return np.sqrt(np.sum(series_log_return**2))
realized_vol = realized_volatility(book_example['log_return'])
print(f'Realized volatility for stock_id 0 on time_id 5 is {realized_vol}')

In [ ]:
import os
from sklearn.metrics import r2_score
import glob
list_order_book_file_train = glob.glob('/kaggle/input/optiver-realized-volatility-prediction/book_train.parquet/*')

In [ ]:
list_order_book_file_train

In [ ]:
def realized_volatility_per_time_id(file_path, prediction_column_name):
    df_book_data = pd.read_parquet(file_path)
    df_book_data['wap'] =(df_book_data['bid_price1'] * df_book_data['ask_size1']+df_book_data['ask_price1'] * df_book_data['bid_size1'])  / (
                                      df_book_data['bid_size1']+ df_book_data[
                                  'ask_size1'])
    df_book_data['log_return'] = df_book_data.groupby(['time_id'])['wap'].apply(log_return)
    df_book_data = df_book_data[~df_book_data['log_return'].isnull()]
    df_realized_vol_per_stock =  pd.DataFrame(df_book_data.groupby(['time_id'])['log_return'].agg(realized_volatility)).reset_index()
    df_realized_vol_per_stock = df_realized_vol_per_stock.rename(columns = {'log_return':prediction_column_name})
    stock_id = file_path.split('=')[1]
    df_realized_vol_per_stock['row_id'] = df_realized_vol_per_stock['time_id'].apply(lambda x:f'{stock_id}-{x}')
    return df_realized_vol_per_stock[['row_id',prediction_column_name]]

In [ ]:
df_book_data1 = pd.read_parquet('/kaggle/input/optiver-realized-volatility-prediction/book_train.parquet/stock_id=97')
df_book_data1
df_book_data1['wap'] =(df_book_data1['bid_price1'] * df_book_data1['ask_size1']+df_book_data1['ask_price1'] * df_book_data1['bid_size1'])  / (
                                      df_book_data1['bid_size1']+ df_book_data1[
                                  'ask_size1'])

In [ ]:
df_book_data1['log_return'] = df_book_data1.groupby(['time_id'])['wap'].apply(log_return)


In [ ]:
df_book_data1 = df_book_data1[~df_book_data1['log_return'].isnull()]
df_realized_vol_per_stock1 =  pd.DataFrame(df_book_data1.groupby(['time_id'])['log_return'].agg(realized_volatility)).reset_index()


In [ ]:
df_realized_vol_per_stock1 = df_realized_vol_per_stock1.rename(columns = {'log_return':'pred'})
df_realized_vol_per_stock1['row_id'] = df_realized_vol_per_stock1['time_id'].apply(lambda x:f'{stock_id}-{x}')


df_realized_vol_per_stock1

In [ ]:
def past_realized_volatility_per_stock(list_file,prediction_column_name):
    df_past_realized = pd.DataFrame()
    for file in list_file:
        df_past_realized = pd.concat([df_past_realized,
                                     realized_volatility_per_time_id(file,prediction_column_name)])
    return df_past_realized

df_past_realized_train = past_realized_volatility_per_stock(list_file=list_order_book_file_train,
                                                           prediction_column_name='pred')


In [ ]:
df_past_realized_train

In [ ]:
train['row_id'] = train['stock_id'].astype(str) + '-' + train['time_id'].astype(str)
train = train[['row_id','target']]


In [ ]:
train

In [ ]:
df_joined = train.merge(df_past_realized_train[['row_id','pred']], on = ['row_id'], how = 'left')

In [ ]:
df_joined

In [ ]:
from sklearn.metrics import r2_score
def rmspe(y_true, y_pred):
    return  (np.sqrt(np.mean(np.square((y_true - y_pred) / y_true))))
R2 = round(r2_score(y_true = df_joined['target'], y_pred = df_joined['pred']),3)
RMSPE = round(rmspe(y_true = df_joined['target'], y_pred = df_joined['pred']),3)
print(f'Performance of the naive prediction: R2 score: {R2}, RMSPE: {RMSPE}')


# Sample Submission

In [ ]:
list_order_book_file_test = glob.glob('/kaggle/input/optiver-realized-volatility-prediction/book_test.parquet/*')
df_naive_pred_test = past_realized_volatility_per_stock(list_file=list_order_book_file_test,
                                                           prediction_column_name='target')
df_naive_pred_test.to_csv('submission.csv',index = False)

In [ ]:


book_train = pd.read_parquet('/kaggle/input/optiver-realized-volatility-prediction/book_train.parquet')
book_test = pd.read_parquet('/kaggle/input/optiver-realized-volatility-prediction/book_test.parquet')
trade_test = pd.read_parquet('/kaggle/input/optiver-realized-volatility-prediction/trade_test.parquet')
trade_train = pd.read_parquet('/kaggle/input/optiver-realized-volatility-prediction/trade_test.parquet')


In [ ]:
train= pd.read_csv("/kaggle/input/optiver-realized-volatility-prediction/train.csv")
test= pd.read_csv("/kaggle/input/optiver-realized-volatility-prediction/test.csv")

In [ ]:
book_train
book_train1=book_train[book_train['stock_id']==0]
book_train2=book_train1[book_train1['time_id']==5]


In [ ]:
book_train2

In [ ]:
book_test

In [ ]:
trade_train

In [ ]:
trade_test

In [ ]:
train

In [ ]:
test